<a href="https://colab.research.google.com/github/MikeDuran-git/Custom_ANN_Model/blob/master/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Libraries


In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow
import keras
import theano

#Uploading the CSV file

In [22]:
from google.colab import files
uploaded= files.upload() # upload the data.csv file

Saving data.csv to data.csv


In [23]:
data=pd.read_csv("data.csv")

In [24]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


#Data Preparation

We remove the first 3 columns sins they do not seem to be useful features.

In [25]:
x=data.iloc[:,3:13].values
x


array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [26]:
y=data.iloc[:,13].values
y

array([1, 0, 1, ..., 1, 1, 0])

##Cathegorical Data handling

We have 2 cathecorical Data insider of our ds, country and gender, lets change them into numbers to make it easier for our model to work with them.

In [27]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer 

label_encoder_X_Country=LabelEncoder()
x[:,1] = label_encoder_X_Country.fit_transform(x[:,1])

label_encoder_X_gender=LabelEncoder()
x[:,2] = label_encoder_X_gender.fit_transform(x[:,2])

# we have 3 countries, lets make a feature for each using the one hot encoder method.
x

array([[619, 0, 0, ..., 1, 1, 101348.88],
       [608, 2, 0, ..., 0, 1, 112542.58],
       [502, 0, 0, ..., 1, 0, 113931.57],
       ...,
       [709, 0, 0, ..., 0, 1, 42085.58],
       [772, 1, 1, ..., 1, 0, 92888.52],
       [792, 0, 0, ..., 1, 0, 38190.78]], dtype=object)

Now we have only numbers insider of our ds 

In [28]:
s=pd.Series(x[:,1])
pd.get_dummies(s).values

new_x=np.delete(x,1,1)
new_x=np.insert(new_x,1,pd.get_dummies(s).values[:,0]
,axis=1)
new_x=np.insert(new_x,2,pd.get_dummies(s).values[:,1]
,axis=1)
new_x

array([[619, 1, 0, ..., 1, 1, 101348.88],
       [608, 0, 0, ..., 0, 1, 112542.58],
       [502, 1, 0, ..., 1, 0, 113931.57],
       ...,
       [709, 1, 0, ..., 0, 1, 42085.58],
       [772, 0, 1, ..., 1, 0, 92888.52],
       [792, 1, 0, ..., 1, 0, 38190.78]], dtype=object)

Now we used the one hot encoding method to transform our cathegorical data into something useful. We keep only 2, to simplify the identification of the ANN, because if there are more than 3 possible outcome it will cause problem.

#Separating the model into training set and Test set.

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test , y_train, y_test = train_test_split(new_x,y,test_size=0.25,random_state = 0)

#Feature Scaling

In [30]:
from sklearn.preprocessing import StandardScaler
standardscaler=StandardScaler()
X_train = standardscaler.fit_transform(X_train)
X_test= standardscaler.fit_transform(X_test)

In [31]:
X_train

array([[-0.73550706, -1.01558815,  1.76021608, ..., -1.55362351,
        -1.03446007, -1.64080994],
       [ 1.02442719,  0.98465111, -0.56811207, ...,  0.64365658,
        -1.03446007, -0.07927152],
       [ 0.80829492, -1.01558815, -0.56811207, ...,  0.64365658,
         0.96668786, -0.99684012],
       ...,
       [ 0.90092304,  0.98465111, -0.56811207, ...,  0.64365658,
        -1.03446007,  1.4133552 ],
       [-0.62229491, -1.01558815, -0.56811207, ...,  0.64365658,
         0.96668786,  0.84496184],
       [-0.28265848, -1.01558815,  1.76021608, ...,  0.64365658,
        -1.03446007,  0.32500428]])

# ANN BUILD

In [32]:
from keras.models import Sequential # to build the model.
from keras.layers import Dense # To build the hidden layers.
from keras.layers import Dropout # this method helps agains overfiting.

let's initialize the ANN with the Sequential method



In [33]:
classifier= Sequential()

let's add a layer. using the Dense method.

- units: how many layers we want to create (in my case i use 6)
- activation: the activation function of this layer (since it is a yes or no case, i use the relu activation functuin).
- kernel_initializer: initialize the weights. (in my case i use uniform)
- Input_dim: The number of features the Entry Layer is gonna have, in my case 11 (to make it more pro i use the command 'X_train.shape[1]')

In [34]:
classifier.add(Dense(units=6,activation='relu',kernel_initializer='uniform',input_dim=X_train.shape[1]))

We add DROPOUT ON EVERY LAYER SO THAT THE RISK OF OVERFITTING IS MINIMAL: I will use a Ratio of 0.1 so that every neuron has a 10% chance of beeing desactivated. (only necessary to build it on the hidden layers and the entry layer.)

In [35]:
classifier.add(Dropout(rate=0.1))

Now we will go deeper, and add a new Hidden Layer. Now we can remove the input_dim argument, since we dont need to tell him the number of features.

In [36]:
classifier.add(Dense(units=6,activation='relu',kernel_initializer='uniform'))
classifier.add(Dropout(rate=0.1))

Now we build the exit layer. The layer that offers the probability we want, since it is a single value we want, we can just offer a single Neuron. and we use the sigmoid activation function because it offers a probability, which is more efficient for a classification case like ours.

In [37]:
classifier.add(Dense(units=1,activation='sigmoid',kernel_initializer='uniform'))

Now we will compile the ANN with the compile method.
- optimizer: algorithme he uses to get the best weights. Using the sga (stocastic gradient algorithme )

- loss: the loss function for the retropropagation, since we do a classification, we need a logistic regression. Thus we use a logarithmic loss function -> the binary_crossentropy.
- metrics: Analyze the perfomance of the model. We use basic accuracy.


In [38]:
classifier.compile(optimizer="adam",loss='binary_crossentropy',metrics=['accuracy'])

#Train the model

Now we need to train the model, using the fit method. Using our training Data, a batch size of 10 (I will modifiy it to see if the accurracy is not that awesome), and 100 epochs.

###hyperparameters


In [39]:
batch_size=30
epochs=100

###training

In [40]:
classifier.fit(X_train,y_train,batch_size=batch_size,epochs=epochs)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.6353 - accuracy: 0.7917
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4513 - accuracy: 0.7973
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4390 - accuracy: 0.7963
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4279 - accuracy: 0.8036
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4278 - accuracy: 0.8011
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4179 - accuracy: 0.8062
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4390 - accuracy: 0.7861
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4401 - accuracy: 0.7923
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4349 - accuracy: 0.7945
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4164 - accura

### let's make some predictions

we will use the classifier method predict on our test x data

In [41]:
y_pred= classifier.predict(X_test)

Since it is visibly not interesting to see the percentage of the results, we use a method to make clients with a percentage higher than 50 % become True and False below.

In [42]:
y_pred = (y_pred >0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

###Making the confusion matrix

to see all the right answers it made, the confusion matrix is very useful.

In [43]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
cm

array([[1952,   39],
       [ 362,  147]])

check the percentage of good answers.

In [44]:
number_of_total_predictions=y_pred.shape[0]

print("The accuracy of our model: "+ str((cm[0][0] + cm[1][1])/ number_of_total_predictions))

The accuracy of our model: 0.8396


# new data

In [45]:
def test_new_client(new_client_list):
  #transform the list into a numpy array 
  new_client=np.array([new_client_list]) #[[]]
  # rescale the data
  scaled_nc=standardscaler.transform(new_client)
  #give the prediction  
  y_new= ( classifier.predict(scaled_nc) > 0.5)
  return y_new[0][0]
  pass

In [46]:
new_client_list=[600,1,0,0,1,3,60000.0,2,1,1,50000.0] # a client that was given with from an exercice.

test_new_client(new_client_list)

False

The the client we just tested is not going to stay in the Bank.

# Optimize the model using K-ford-Cross validation

We use the scikitlearn lib to use the crooss validation with keras.


In [62]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

we make a function to build a default classifier. Because the keras classifier needs a function to build it.

In [63]:
def build_classifier():
  classifier= Sequential()
  classifier.add(Dense(units=6,activation='relu',kernel_initializer='uniform',input_dim=X_train.shape[1]))
  classifier.add(Dense(units=6,activation='relu',kernel_initializer='uniform'))
  classifier.add(Dense(units=1,activation='sigmoid',kernel_initializer='uniform'))
  classifier.compile(optimizer="adam",loss='binary_crossentropy',metrics=['accuracy'])
  return classifier

In [64]:
#hyperparameters
batch_size=30
epochs=500
cv=10 # the number of k validations we do.

In [65]:
classifier= KerasClassifier(build_fn=build_classifier,batch_size=batch_size,epochs=epochs)

Now the classifier is made, we need to get the values of all the cross validations. using the method cross_val_score :
- estimator: our classifier
- X and Y : training data
- cv: the number of K in k-ford we use to build our model (in our case i use 10 because it seems to be reasonable.)
- n_jobs : the number of CPU that work on it, i say 1 because now all my cpu power works on it, which makes it faster.

In [66]:
precisions= cross_val_score(estimator=classifier,X=X_train,y=y_train,cv=cv, n_jobs=-1)

In [67]:
mean=precisions.mean()
std=precisions.std()

In [68]:
mean

0.8448000073432922

In [69]:
std

0.01737636094458025

#Test a optimisation with Gridsearch

I will use the GridsearchCV method, to use multiple params and also see if the classifier work with multiple hyperparameters.

In [52]:
from sklearn.model_selection import GridSearchCV

I make a new classifier function that takes an optimizer, so that i can vary without writing new code.

In [53]:
def build_classifier(optimizer):
  classifier= Sequential()
  classifier.add(Dense(units=6,activation='relu',kernel_initializer='uniform',input_dim=X_train.shape[1]))
  classifier.add(Dense(units=6,activation='relu',kernel_initializer='uniform'))
  classifier.add(Dense(units=1,activation='sigmoid',kernel_initializer='uniform'))
  classifier.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
  return classifier

I will make a new classifier.


In [54]:
classifier=KerasClassifier(build_fn=build_classifier)

I make a dictionary to have multiple values for the keys i will use. I will also change the optimizer into rmsprop


In [55]:
parameters = {'batch_size': [25,32],'epochs': [100,500],'optimizer':["adam","rmsprop"]}

Now we use a gridsearchcv method with all our parameters.

In [56]:
grid_search=GridSearchCV(estimator=classifier,
                        param_grid=parameters,
                         scoring="accuracy",
                         cv=cv)


let's train our new model

In [ ]:
grid_search=grid_search.fit(X_train,y_train)

that took 3 hours..... my god..

lets analyze our result.

In [60]:
grid_search.best_params_

{'batch_size': 25, 'epochs': 500, 'optimizer': 'rmsprop'}

In [61]:
grid_search.best_score_

0.8495999999999999

Now we know which parameters to train our model are the best !!!! 